In [1]:

from tqdm import tqdm
import tensorflow as tf
import json
import tensorflow_text as text
import os
import numpy as np
import pandas as pd
import pickle
from matchms import set_matchms_logger_level
import pandas as pd
set_matchms_logger_level("ERROR")
from matchms.filtering import add_losses
from matchms.filtering import add_parent_mass
from matchms.filtering import default_filters
from matchms.filtering import normalize_intensities
from matchms.filtering import select_by_intensity
from matchms.filtering import reduce_to_number_of_peaks
from matchms.filtering import require_minimum_number_of_peaks
from matchms.filtering import select_by_mz
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
from matchms.importing import load_from_msp


from matchms.filtering import repair_inchi_inchikey_smiles
from matchms.filtering import derive_inchikey_from_inchi
from matchms.filtering import derive_smiles_from_inchi
from matchms.filtering import derive_inchi_from_smiles
from matchms.filtering import harmonize_undefined_inchi
from matchms.filtering import harmonize_undefined_inchikey
from matchms.filtering import harmonize_undefined_smiles

In [5]:
!python -V

Python 3.9.15


In [2]:
%%time

#from spec2vec.model_building import train_new_word2vec_model

def spectrum_processing(s):
    """This is how one would typically design a desired pre- and post-
    processing pipeline."""
    s = default_filters(s)
    s = add_parent_mass(s)
    s = normalize_intensities(s)
    s = select_by_intensity(s, intensity_from=0.01)
    s = reduce_to_number_of_peaks(s, n_required=5, n_max=250)
    s = select_by_mz(s, mz_from=15, mz_to=2000)
    s = add_losses(s, loss_mz_from=15.0, loss_mz_to=350.0)
    s = require_minimum_number_of_peaks(s, n_required=5)
    return s



def metadata_processing(spectrum):
    spectrum = default_filters(spectrum)
    spectrum = repair_inchi_inchikey_smiles(spectrum)
    spectrum = derive_inchi_from_smiles(spectrum)
    spectrum = derive_smiles_from_inchi(spectrum)
    spectrum = derive_inchikey_from_inchi(spectrum)
    spectrum = harmonize_undefined_smiles(spectrum)
    spectrum = harmonize_undefined_inchi(spectrum)
    spectrum = harmonize_undefined_inchikey(spectrum)
    return spectrum
# Load data from MGF file and apply filters

import os
from matchms.importing import load_from_mgf
path_data = "C:/Users/delser/mass2smiles"  # enter path to downloaded mgf file
file_mgf = os.path.join(path_data, 
                        "casmi_candidates_pos_casmi_id.mgf")
spectrums = list(load_from_mgf(file_mgf))

spectrums = [metadata_processing(s) for s in spectrums]
spectrums = [spectrum_processing(s) for s in spectrums]
#spectrums = [spectrum_processing(s) for s in load_from_mgf("/Users/delser/Desktop/PhD/Phytochemistry/NP-Databases/CFM-4_DB/TOTAL_COMPOUNDS_DB.energies_merged_name.mgf")]
#spectrums = [spectrum_processing(s) for s in load_from_mgf("/Users/delser/Desktop/PhD/Phytochemistry/FBMN/alltissues/altissues15072021-py.mgf")]
# Omit spectrums that didn't qualify for analysis
spectrums = [s for s in spectrums if s is not None]


# Create spectrum documents
#reference_documents = [SpectrumDocument(s, n_decimals=2) for s in spectrums]

CPU times: total: 7.12 s
Wall time: 7.25 s


In [3]:
precs = []
IDs = []
mzs=[]
ints=[]
loss_mzs=[]
loss_ints=[]


for spec in spectrums: 
    IDs.append(spec.get("feature_id"))
    precs.append(spec.get("precursor_mz"))
    mzs.append(list(spec.peaks.mz))
    ints.append(list(spec.peaks.intensities))
    loss_mzs.append(list(spec.losses.mz))
    loss_ints.append(list(spec.losses.intensities))
    

In [27]:
res = dict(zip(mzs[3]+loss_mzs[3], ints[3]+loss_ints[3]))
res

{56.604: 0.020625,
 57.0337: 0.03625,
 61.0286: 0.10625,
 69.0336: 0.51875,
 71.0492: 0.2,
 74.5668: 0.018125,
 81.0336: 0.075,
 83.0491: 0.3,
 85.0283: 0.425,
 87.044: 0.1625,
 95.0492: 0.024375,
 97.0286: 0.125,
 99.0442: 0.034375,
 109.0281: 0.0325,
 111.0441: 0.1625,
 115.0384: 0.02625,
 127.0392: 0.026875,
 129.0546: 0.1125,
 232.1556: 0.020625,
 299.0545: 1.0,
 299.0909: 0.036875,
 311.0572: 0.01875,
 355.1155: 0.15,
 391.0793: 0.0225,
 407.7533: 0.02,
 498.4384: 0.02125,
 206.79970000000003: 0.02125,
 297.4848: 0.02,
 314.15880000000004: 0.0225}

In [28]:
res=dict(sorted(res.items()))
res

{56.604: 0.020625,
 57.0337: 0.03625,
 61.0286: 0.10625,
 69.0336: 0.51875,
 71.0492: 0.2,
 74.5668: 0.018125,
 81.0336: 0.075,
 83.0491: 0.3,
 85.0283: 0.425,
 87.044: 0.1625,
 95.0492: 0.024375,
 97.0286: 0.125,
 99.0442: 0.034375,
 109.0281: 0.0325,
 111.0441: 0.1625,
 115.0384: 0.02625,
 127.0392: 0.026875,
 129.0546: 0.1125,
 206.79970000000003: 0.02125,
 232.1556: 0.020625,
 297.4848: 0.02,
 299.0545: 1.0,
 299.0909: 0.036875,
 311.0572: 0.01875,
 314.15880000000004: 0.0225,
 355.1155: 0.15,
 391.0793: 0.0225,
 407.7533: 0.02,
 498.4384: 0.02125}

In [32]:
type(list(res.keys()))

list

In [30]:
res.values()

dict_values([0.020625, 0.03625, 0.10625, 0.51875, 0.2, 0.018125, 0.075, 0.3, 0.425, 0.1625, 0.024375, 0.125, 0.034375, 0.0325, 0.1625, 0.02625, 0.026875, 0.1125, 0.02125, 0.020625, 0.02, 1.0, 0.036875, 0.01875, 0.0225, 0.15, 0.0225, 0.02, 0.02125])

In [4]:
metadata = pd.DataFrame(list(zip(IDs, precs,mzs,ints,loss_mzs,loss_ints)), columns=["feature_id", "precursor_mz","mzs","intensities","loss_mzs","loss_intensities" ])
metadata

,feature_id,precursor_mz,mzs,intensities,loss_mzs,loss_intensities
0,398,235.1691,"[53.0389, 53.0509, 55.0181, 55.0545, 55.5835, ...","[0.07166666666666667, 0.020833333333333332, 0....","[19.8476, 19.87299999999999, 56.06339999999997...","[0.075, 0.07083333333333333, 0.031666666666666..."
1,398,235.1691,"[53.0389, 53.0509, 55.0181, 55.0545, 55.5835, ...","[0.07166666666666667, 0.020833333333333332, 0....","[19.8476, 19.87299999999999, 56.06339999999997...","[0.075, 0.07083333333333333, 0.031666666666666..."
2,159,485.2164,"[66.3952, 68.416, 81.3048, 100.9429, 121.0081,...","[0.05, 0.04565217391304348, 0.0521739130434782...","[18.01230000000004, 96.17940000000004, 102.068...","[1.0, 0.058695652173913045, 0.0630434782608695..."
3,169,705.2381,"[56.604, 57.0337, 61.0286, 69.0336, 71.0492, 7...","[0.020625, 0.03625, 0.10625, 0.51875, 0.2, 0.0...","[206.79970000000003, 297.4848, 314.15880000000...","[0.02125, 0.02, 0.0225]"
4,423,441.2264,"[62.099, 62.3016, 65.6795, 71.5638, 95.0857, 1...","[0.07096774193548387, 0.08709677419354839, 0.0...","[70.07659999999998, 88.08980000000003, 130.173...","[1.0, 0.14516129032258066, 0.12903225806451613..."
...,...,...,...,...,...,...
231,439,336.0647,"[55.173, 55.6729, 57.7501, 63.2992, 64.3001, 7...","[0.010476190476190476, 0.010952380952380953, 0...","[17.859800000000007, 72.62130000000002, 119.40...","[0.04428571428571428, 0.010952380952380953, 0...."
232,322,532.3113,"[53.6431, 61.0821, 67.0543, 69.0334, 81.0699, ...","[0.0125, 0.012, 0.016, 0.08, 0.23, 0.285, 0.07...","[197.09179999999998, 207.04439999999994, 215.1...","[0.0245, 0.015, 0.205, 0.135, 0.28, 0.0205, 0...."
233,300,478.3159,"[63.7647, 71.5594, 93.5641, 108.7938, 119.8244...","[0.10476190476190476, 0.1, 0.11428571428571428...","[16.459400000000016, 35.03750000000002, 53.050...","[0.10952380952380952, 0.13333333333333333, 0.1..."
234,254,329.2320,"[53.0389, 55.0182, 55.0546, 67.008, 67.0544, 6...","[0.0456140350877193, 0.05087719298245614, 0.28...","[89.70510000000002, 112.67550000000003, 145.15...","[0.042105263157894736, 0.2631578947368421, 0.0..."


In [5]:
df_train=metadata.dropna()
df_train

,feature_id,precursor_mz,mzs,intensities,loss_mzs,loss_intensities
0,398,235.1691,"[53.0389, 53.0509, 55.0181, 55.0545, 55.5835, ...","[0.07166666666666667, 0.020833333333333332, 0....","[19.8476, 19.87299999999999, 56.06339999999997...","[0.075, 0.07083333333333333, 0.031666666666666..."
1,398,235.1691,"[53.0389, 53.0509, 55.0181, 55.0545, 55.5835, ...","[0.07166666666666667, 0.020833333333333332, 0....","[19.8476, 19.87299999999999, 56.06339999999997...","[0.075, 0.07083333333333333, 0.031666666666666..."
2,159,485.2164,"[66.3952, 68.416, 81.3048, 100.9429, 121.0081,...","[0.05, 0.04565217391304348, 0.0521739130434782...","[18.01230000000004, 96.17940000000004, 102.068...","[1.0, 0.058695652173913045, 0.0630434782608695..."
3,169,705.2381,"[56.604, 57.0337, 61.0286, 69.0336, 71.0492, 7...","[0.020625, 0.03625, 0.10625, 0.51875, 0.2, 0.0...","[206.79970000000003, 297.4848, 314.15880000000...","[0.02125, 0.02, 0.0225]"
4,423,441.2264,"[62.099, 62.3016, 65.6795, 71.5638, 95.0857, 1...","[0.07096774193548387, 0.08709677419354839, 0.0...","[70.07659999999998, 88.08980000000003, 130.173...","[1.0, 0.14516129032258066, 0.12903225806451613..."
...,...,...,...,...,...,...
231,439,336.0647,"[55.173, 55.6729, 57.7501, 63.2992, 64.3001, 7...","[0.010476190476190476, 0.010952380952380953, 0...","[17.859800000000007, 72.62130000000002, 119.40...","[0.04428571428571428, 0.010952380952380953, 0...."
232,322,532.3113,"[53.6431, 61.0821, 67.0543, 69.0334, 81.0699, ...","[0.0125, 0.012, 0.016, 0.08, 0.23, 0.285, 0.07...","[197.09179999999998, 207.04439999999994, 215.1...","[0.0245, 0.015, 0.205, 0.135, 0.28, 0.0205, 0...."
233,300,478.3159,"[63.7647, 71.5594, 93.5641, 108.7938, 119.8244...","[0.10476190476190476, 0.1, 0.11428571428571428...","[16.459400000000016, 35.03750000000002, 53.050...","[0.10952380952380952, 0.13333333333333333, 0.1..."
234,254,329.2320,"[53.0389, 55.0182, 55.0546, 67.008, 67.0544, 6...","[0.0456140350877193, 0.05087719298245614, 0.28...","[89.70510000000002, 112.67550000000003, 145.15...","[0.042105263157894736, 0.2631578947368421, 0.0..."


df_train=df_train.loc[df_train.loss_mzs.apply(str) != "[]"]
df_train

df_wrong=metadata.loc[set(metadata.index) - set(df_train.index.values.tolist())]
df_wrong

In [6]:
df_train.to_csv('casmi_ids.tsv',sep='\t')

In [33]:
df_train = pd.read_csv("casmi_func_groups_2201.tsv",sep="\t")
#data_df = pd.read_csv("/Users/delser/mass2smiles/retrain/nist/all_HRMS_validation_16122022_cddd_refine.tsv",sep="\t")
df_train=df_train.dropna()
df_train

,Unnamed: 0,spectrum_id,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,smiles_preprocessed,num_of_sugars,Number of aliphatic carboxylic acids,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
0,0,3,719.2538,"[53.0387, 55.0179, 55.0545, 57.0338, 59.0492, ...","[0.017272727272727273, 0.01818181818181818, 0....",[314.15709999999996],[0.01818181818181818],CC1C(C(C(C(O1)OC2=C(OC3=C(C(=CC(=C3C2=O)O)O)CC...,2.0,0,...,2,0,0,0,0,0,0,0,0,0
1,1,5,499.2298,"[67.0543, 69.0698, 81.0699, 83.0492, 83.0853, ...","[0.010357142857142856, 0.04642857142857143, 0....","[60.02260000000001, 118.02660000000003, 160.03...","[0.060714285714285714, 0.39285714285714285, 0....",CC1CC2(C(C1O)C=C(C(CC3C(C3(C)C)C=C(C2=O)C)OC(=...,0.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,6,1102.5777,"[81.0334, 83.0491, 85.0224, 85.0284, 85.0333, ...","[0.14210526315789473, 0.2236842105263158, 0.01...",[],[],CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OCC3C(C(C(C(O3)O...,4.0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,10,472.2082,"[72.2736, 145.0759, 148.0868, 149.0707, 172.06...","[0.010697674418604652, 0.03488372093023256, 0....","[135.07919999999996, 148.0632, 165.08959999999...","[0.053488372093023255, 0.10465116279069768, 0....",CN1C2=C(C=C(C=C2)C(=O)N(CCC(=O)O)C3=CC=CC=N3)N...,0.0,1,...,0,0,0,0,0,1,0,0,0,0
4,4,11,657.3116,"[55.0542, 55.9818, 57.0334, 60.2456, 69.0335, ...","[0.018260869565217393, 0.013043478260869565, 0...","[158.094, 210.1282, 228.1352, 246.1485, 280.16...","[0.02217391304347826, 0.06086956521739131, 0.1...",CCCC(=O)OCC(C(C(CN1C2=C(C=C(C(=C2)C)C)N=C3C1=N...,1.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,237,490,268.1541,"[86.0599, 109.065, 109.1013, 121.101, 123.1168...","[0.10689655172413794, 0.04482758620689655, 0.0...","[18.01030000000003, 36.02120000000002, 46.0054...","[0.21724137931034482, 0.06206896551724138, 1.0...",CC1CCC2C1C(=O)OC(C2C)NC(=O)C3C(O3)C,0.0,0,...,0,0,0,0,0,0,0,0,0,0
238,238,491,411.3254,"[55.0544, 57.0701, 67.0544, 69.0699, 81.0699, ...","[0.0215625, 0.053125, 0.015, 0.34375, 0.046875...","[18.01060000000001, 88.08850000000001, 142.135...","[0.153125, 0.02125, 0.01875, 0.02625, 0.028437...",CC(C)C(C)C=CC(C)C1CCC2C1(CCC3=C2C(=O)C=C4C3(CC...,0.0,0,...,0,0,0,0,0,0,0,0,0,0
239,239,492,430.2432,"[50.4192, 52.9216, 74.1355, 81.0699, 86.4846, ...","[0.05555555555555555, 0.058333333333333334, 0....","[98.63150000000002, 179.0794, 197.0896, 214.55...","[0.06666666666666667, 0.2777777777777778, 1.0,...",CC1CCC(C2(C1=CC(CC2)C(=C)C(=O)O)C)OC3C(C(C(C(O...,1.0,1,...,0,0,0,0,0,0,0,0,0,0
240,240,495,578.2076,"[54.5814, 57.0338, 61.0286, 69.0335, 81.0331, ...","[0.01, 0.010769230769230769, 0.038461538461538...",[160.76359999999994],[0.011923076923076923],COC1=C2C(=CC(=C1OC3C(C(C(C(O3)CO)O)O)OC4C(C(C(...,2.0,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_wrong.to_csv('loss_fail_matchms.tsv',sep='\t')

In [34]:
def positional_encoding(max_position, d_model, min_freq=1e-6):
    position = np.arange(max_position)
    freqs = min_freq**(2*(np.arange(d_model)//2)/d_model)
    pos_enc = position.reshape(-1,1)*freqs.reshape(1,-1)
    pos_enc[:, ::2] = np.cos(pos_enc[:, ::2])
    pos_enc[:, 1::2] = np.sin(pos_enc[:, 1::2])
    return pos_enc

def trun_n_d(n,d):
    return (  n if not n.find('.')+1 else n[:n.find('.')+d+1]  )

In [35]:
P=positional_encoding(200000,256, min_freq=1e2)
#np.save('positions_512_1e2.npy',P)

In [11]:
#matchms mgf encoding

def prepro_specs_train(df):
    valid=[]
    precs=df['precursor_mz'].to_list()
    mzs=df['mzs'].to_list()
    ints=df['intensities'].to_list()
    loss_mzs=df['loss_mzs'].to_list()
    loss_ints=df['loss_intensities'].to_list()
    for one_pre,one_mzs,one_ints,one_loss,one_loss_ints in tqdm(zip(precs,mzs,ints,loss_mzs,loss_ints)):
        mz_list=[round(float(trun_n_d(str(one_pre),2))*100)] # add precursor mz
        intes_list=[2.0] # add precursor int
        res = dict(zip(one_mzs+one_loss, one_ints+one_loss_ints))  # order by mzs
        res=dict(sorted(res.items()))
        for m,i in zip(list(res.keys()), list(res.values())): # change this from mgf from matchms
            mz=round(float(trun_n_d(str(m),2))*100)
            mz_list.append(mz)
            intens=round(i,4)
            intes_list.append(intens)
        int_mzs=[intes_list,mz_list]   
        valid.append(int_mzs) # put intesities at first
    return tf.ragged.constant(valid)
        

In [36]:
def prepro_specs_train(df):
    valid=[]
    precs=df['precursor_mz'].to_list()
    mzs=df['mzs'].to_list()
    ints=df['intensities'].to_list()
    loss_mzs=df['loss_mzs'].to_list()
    loss_ints=df['loss_intensities'].to_list()
    for one_pre,one_mzs,one_ints,one_loss,one_loss_ints in tqdm(zip(precs,mzs,ints,loss_mzs,loss_ints)):
        mz_list=[round(float(trun_n_d(str(one_pre),2))*100)] # add precursor mz
        intes_list=[2.0] # add precursor int
        res = dict(zip(json.loads(one_mzs)+json.loads(one_loss), json.loads(one_ints)+json.loads(one_loss_ints)))  # order by mzs
        res=dict(sorted(res.items()))
        for m,i in zip(list(res.keys()), list(res.values())): # change this from mgf from matchms
            mz=round(float(trun_n_d(str(m),2))*100)
            mz_list.append(mz)
            intens=round(i,4)
            intes_list.append(intens)
        int_mzs=[intes_list,mz_list]   
        valid.append(int_mzs) # put intesities at first
    return tf.ragged.constant(valid)

In [37]:
%%time
train=prepro_specs_train(df_train)

242it [00:00, 1234.65it/s]


CPU times: total: 516 ms
Wall time: 475 ms


In [17]:
tf.gather(train, [0, 1, 2, 3, 4]).shape

TensorShape([5, None, None])

In [33]:
train[0:32].shape

TensorShape([32, None, None])

In [26]:
length=[i[0].shape[0] for i in train]
max(length)

253

In [38]:
dimn=256
def encoding(rag_tensor,P,dimn):
    to_pad=[]
    for sample in rag_tensor:
        all_dim=[sample[0].numpy().tolist()]
        pos_enc=[P[int(i)-1] for i in sample[1].numpy().tolist()]
        for dim in range(dimn):
            dim_n=[i[dim] for i in pos_enc]
            all_dim.append(dim_n)
        to_pad.append(all_dim)
    to_pad=[tf.keras.preprocessing.sequence.pad_sequences(i,maxlen=501,dtype='float32',padding='post',truncating='post',value=10) for i in to_pad]
    to_pad=np.stack((to_pad))
    to_pad=np.swapaxes(to_pad, 1, -1)
    return to_pad

In [38]:
xtrain[0][1]

array([ 0.0717    ,  0.5332156 , -0.8459794 ,  0.01143887, -0.99993455,
        0.28827846, -0.95754665,  0.9954703 ,  0.09507313, -0.95233387,
        0.3050577 ,  0.59969896,  0.8002257 ,  0.57598245,  0.8174621 ,
       -0.996885  , -0.07886912, -0.17545353,  0.9844877 , -0.9999982 ,
       -0.00188863,  0.1089685 ,  0.9940452 , -0.98186713, -0.18957031,
       -0.95267904,  0.30397803,  0.9439646 ,  0.33004668, -0.80919385,
        0.58754176, -0.9424226 , -0.3344243 , -0.86887133, -0.49503803,
       -0.94475156, -0.32778722, -0.9721955 , -0.2341707 , -0.7721371 ,
       -0.63545597,  0.5649307 , -0.82513833, -0.4526213 ,  0.89170283,
        0.68732804,  0.72634715, -0.56023455,  0.82833403,  0.93247634,
        0.36123106, -0.654053  ,  0.75644875, -0.6053066 ,  0.7959924 ,
        0.69717646,  0.7168996 ,  0.22841789, -0.9735632 , -0.92402524,
        0.38233152,  0.37131587,  0.9285066 ,  0.55711097,  0.8304381 ,
       -0.9599877 ,  0.28004214,  0.25998947,  0.96561146, -0.96

In [39]:
%%time
xtrain=encoding(train,P,dimn)

CPU times: total: 7.39 s
Wall time: 7.39 s


In [40]:
%%time
np.save('casmi_specs.npy',xtrain)

CPU times: total: 3.28 s
Wall time: 3.29 s
